<a href="https://colab.research.google.com/github/basketbob/ML/blob/main/netology_DS/numpy_33_lesson4_kuzinvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

#Зададание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений


In [ ]:
def source_type(row):
  source = None
  ad = ['email', 'paid']

  if row.traffic_source == 'yandex' or row.traffic_source == 'google':
    source = 'organic'
  elif row.traffic_source in ad:
    if row.region == 'Russia':
      source = 'ad'
    else:
      source = 'other'

  return source


logs = pd.read_csv('visit_log.csv', sep=';')
logs['source_type'] = logs.apply(source_type, axis=1)
logs.head(100)


#Зададание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
urls = pd.read_csv('URLs.txt')
urls[urls['url'].str.contains(r'\/\d{8}-')].head(20)

#Зададание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [ ]:
ratings = pd.read_csv('ratings.csv')
#группируем по пользователям
ratings = ratings.groupby('userId')
#вычисляем количество оценок, максимальное и минимальное значение времени
ratings = ratings.agg(mx=('timestamp', 'max'), mn=('timestamp', 'min'), cnt=('rating', 'count'))
#вычисляем "время жизни" пользователя
ratings.loc[ratings.cnt > 100, 'lifetime'] = ratings.mx - ratings.mn
ratings.head(20)

,mx,mn,cnt,lifetime
userId,,,,
1,1260759205,1260759108,20,NaN
2,835356246,835355395,76,NaN
3,1298932787,1298861589,51,NaN
4,949982274,949778714,204,203560.0
5,1163375145,1163373044,100,NaN
6,1109258285,1108134263,44,NaN
7,851869291,851866703,88,NaN
8,1154474527,1154389340,116,85187.0
9,939122916,938627748,45,NaN


#Зададание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента


In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

#добавляем столбец со способом перевозки клиентов
rzd['source'] = 'rzd'
auto['source'] = 'auto'
air['source'] = 'air'

#переименовываем в 3 таблицах столбец с дохом в одинаковое имя
rzd.rename(columns={'rzd_revenue': 'revenue'}, inplace=True)
auto.rename(columns={'auto_revenue': 'revenue'}, inplace=True)
air.rename(columns={'air_revenue': 'revenue'}, inplace=True)

#соединяем три таблицы по типу OUTER
traffic = pd.concat([rzd, auto, air])

#группируем данные с помощью сводной таблицы
traffic = traffic.pivot_table(index='client_id', columns='source', values='revenue', aggfunc='sum', fill_value=0).reset_index()

#добавляем в сводную таблицу адреса клиентов
traffic = traffic.merge(client_base, on='client_id', how='left')

#смотрим что получилось
traffic.head(20)

,client_id,air,auto,rzd,address
0,111,0,0,1093,Комсомольская 4
1,112,0,0,2810,Энтузиастов 8а
2,113,0,57483,10283,Левобережная 1а
3,114,0,83,5774,Мира 14
4,115,81,912,981,ЗЖБИиДК 1
5,116,4,4834,0,Строителей 18
6,117,13,98,0,Панфиловская 33
7,118,173,0,0,Мастеркова 4
